In [9]:
# Loading Libraries

# Set environment params
Sys.setenv(LANG='en')  # English

# Import libraries
library(data.table)  # Data manipulate
library(mlr)         # ML toolkit
library(caret)       # ML toolkit
library(ggplot2)     # Visualization
library(pROC)        # AUC, ROC
library(e1071)       # SVM
library(tidyverse)

# Import data
library(ISLR)      # Data from the course book
library(MASS)      # Boston housing dataset
library(datasets)  # US crime dataset
library(tidyverse)
library(leaps)

# Resize plot
library(repr)  # String and binary representations
#options(repr.plot.width=4, repr.plot.height=4)
#par(cex.lab=0.8, cex.axis=0.8, cex.main=0.8, cex.sub=0.8)


Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last


The following object is masked from 'package:purrr':

    transpose


Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:mlr':

    train


The following object is masked from 'package:purrr':

    lift


Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var



Attaching package: 'e1071'


The following object is masked from 'package:mlr':

    impute



Attaching package: 'MASS'


The following object is masked from 'package:dplyr':



In [26]:
# Loading data
bank<-read.csv("./data/bankruptcy_prediction/data.csv")

In [4]:
head(bank)

,Bankrupt.,ROA.C..before.interest.and.depreciation.before.interest,ROA.A..before.interest.and...after.tax,ROA.B..before.interest.and.depreciation.after.tax,Operating.Gross.Margin,Realized.Sales.Gross.Margin,Operating.Profit.Rate,Pre.tax.net.Interest.Rate,After.tax.net.Interest.Rate,Non.industry.income.and.expenditure.revenue,⋯,Net.Income.to.Total.Assets,Total.assets.to.GNP.price,No.credit.Interval,Gross.Profit.to.Sales,Net.Income.to.Stockholder.s.Equity,Liability.to.Equity,Degree.of.Financial.Leverage..DFL.,Interest.Coverage.Ratio..Interest.expense.to.EBIT.,Net.Income.Flag,Equity.to.Liability
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,1,0.3705943,0.4243894,0.4057498,0.6014572,0.6014572,0.9989692,0.7968871,0.8088094,0.3026464,⋯,0.7168453,0.009219440,0.6228790,0.6014533,0.8278902,0.2902019,0.02660063,0.5640501,1,0.01646874
2,1,0.4642909,0.5382141,0.5167300,0.6102351,0.6102351,0.9989460,0.7973802,0.8093007,0.3035564,⋯,0.7952971,0.008323302,0.6236517,0.6102365,0.8399693,0.2838460,0.26457682,0.5701749,1,0.02079431
3,1,0.4260713,0.4990188,0.4722951,0.6014500,0.6013635,0.9988574,0.7964034,0.8083875,0.3020352,⋯,0.7746697,0.040002853,0.6238410,0.6014493,0.8367743,0.2901885,0.02655472,0.5637061,1,0.01647411
4,1,0.3998440,0.4512647,0.4577333,0.5835411,0.5835411,0.9986997,0.7969670,0.8089656,0.3033495,⋯,0.7395545,0.003252475,0.6229287,0.5835376,0.8346971,0.2817212,0.02669663,0.5646634,1,0.02398233
5,1,0.4650222,0.5384322,0.5222978,0.5987835,0.5987835,0.9989731,0.7973661,0.8093037,0.3034750,⋯,0.7950159,0.003877563,0.6235207,0.5987815,0.8399727,0.2785138,0.02475185,0.5756166,1,0.03549020
6,1,0.3886803,0.4151766,0.4191338,0.5901714,0.5902507,0.9987581,0.7969032,0.8087706,0.3031158,⋯,0.7104205,0.005277875,0.6226046,0.5901723,0.8299390,0.2850871,0.02667537,0.5645383,1,0.01953448


In [11]:
# Data description
str(bank)

'data.frame':	6819 obs. of  96 variables:
 $ Bankrupt.                                              : int  1 1 1 1 1 1 0 0 0 0 ...
 $ ROA.C..before.interest.and.depreciation.before.interest: num  0.371 0.464 0.426 0.4 0.465 ...
 $ ROA.A..before.interest.and...after.tax                 : num  0.424 0.538 0.499 0.451 0.538 ...
 $ ROA.B..before.interest.and.depreciation.after.tax      : num  0.406 0.517 0.472 0.458 0.522 ...
 $ Operating.Gross.Margin                                 : num  0.601 0.61 0.601 0.584 0.599 ...
 $ Realized.Sales.Gross.Margin                            : num  0.601 0.61 0.601 0.584 0.599 ...
 $ Operating.Profit.Rate                                  : num  0.999 0.999 0.999 0.999 0.999 ...
 $ Pre.tax.net.Interest.Rate                              : num  0.797 0.797 0.796 0.797 0.797 ...
 $ After.tax.net.Interest.Rate                            : num  0.809 0.809 0.808 0.809 0.809 ...
 $ Non.industry.income.and.expenditure.revenue            : num  0.303 0.304 0.30

Q1) Randomly divide data into train/test as 80/20 (set.seed=1).

In [27]:
# Split the data into training and test set
set.seed(1)
tra.samples <- bank$Bankrupt. %>% 
  createDataPartition(p = 0.8, list = FALSE)
train.data  <- bank[tra.samples, ]
test.data <- bank[-tra.samples, ]

In [23]:
str(train.data)

'data.frame':	5456 obs. of  96 variables:
 $ Bankrupt.                                              : int  1 1 1 1 0 0 0 0 0 0 ...
 $ ROA.C..before.interest.and.depreciation.before.interest: num  0.371 0.426 0.4 0.389 0.391 ...
 $ ROA.A..before.interest.and...after.tax                 : num  0.424 0.499 0.451 0.415 0.446 ...
 $ ROA.B..before.interest.and.depreciation.after.tax      : num  0.406 0.472 0.458 0.419 0.436 ...
 $ Operating.Gross.Margin                                 : num  0.601 0.601 0.584 0.59 0.62 ...
 $ Realized.Sales.Gross.Margin                            : num  0.601 0.601 0.584 0.59 0.62 ...
 $ Operating.Profit.Rate                                  : num  0.999 0.999 0.999 0.999 0.999 ...
 $ Pre.tax.net.Interest.Rate                              : num  0.797 0.796 0.797 0.797 0.797 ...
 $ After.tax.net.Interest.Rate                            : num  0.809 0.808 0.809 0.809 0.809 ...
 $ Non.industry.income.and.expenditure.revenue            : num  0.303 0.302 0.303 

Q2) Build a NN model with 1 hidden layer of 30 neurons, sigmoid activation function

In [28]:
# Create the formula
bank_formula <- as.formula("Bankrupt. ~ Operating.Gross.Margin + Operating.Profit.Rate + Cash.flow.rate + Current.Ratio + Borrowing.dependency + Revenue.per.person") 

In [35]:
# Train the neural network model
nn_md <- neuralnet(bank_formula,          # The formula
                   train.data,            # Training data
                   hidden=30,             # Size of the hidden layers
                   stepmax=10000,         # Maximum training step before stopping
                   rep=30,                # Number of training repeat
                   lifesign='full',       # Print during train
                   algorithm='backprop',  # Algorithm to calculate the network (backpropagation)
                   learningrate=0.01,     # Learning rate
                   err.fct='ce',          # Error function, cross-entropy
                   act.fct="logistic",    # Function use to calculate the result (activation function)
                   linear.output=F        # FALSE = apply activation function to output layer
                  )

hidden: 30    thresh: 0.01    rep:  1/30    steps: 
   1000	min thresh: 65.3439637998688
                                                   
   2000	min thresh: 65.3439637998688
                                                   
   3000	min thresh: 5.84092605225384
                                                   
   4000	min thresh: 3.70733732049258
                                                   
   5000	min thresh: 3.70733732049258
                                                   
   6000	min thresh: 1.90691358673313
                                                   
   7000	min thresh: 1.90691358673313
                                                   
   8000	min thresh: 1.90691358673313
                                                   
   9000	min thresh: 1.90691358673313
                                                   
stepmax	min thresh: 1.90691358673313

hidden: 30    thresh: 0.01    rep:  2/30    steps: 
   1000	min thresh: 17.930443143887
                     

   3000	min thresh: 13.4447498370693
                                                   
   4000	min thresh: 3.66830281742567
                                                   
   5000	min thresh: 3.66830281742567
                                                   
   6000	min thresh: 3.66830281742567
                                                   
   7000	min thresh: 3.66830281742567
                                                   
   8000	min thresh: 3.09337483831261
                                                   
   9000	min thresh: 3.09337483831261
                                                   
stepmax	min thresh: 3.09337483831261

hidden: 30    thresh: 0.01    rep: 11/30    steps: 
   1000	min thresh: 6.16465199838066
                                                   
   2000	min thresh: 6.16465199838066
                                                   
   3000	min thresh: 6.16465199838066
                                                   
   4000	min thresh: 

   5000	min thresh: 23.2577516818601
                                                   
   6000	min thresh: 1.78425415186313
                                                   
   7000	min thresh: 1.78425415186313
                                                   
   8000	min thresh: 1.78425415186313
                                                   
   9000	min thresh: 1.78425415186313
                                                   
stepmax	min thresh: 1.78425415186313

hidden: 30    thresh: 0.01    rep: 20/30    steps: 
   1000	min thresh: 80.6963435594134
                                                   
   2000	min thresh: 80.6963435594134
                                                   
   3000	min thresh: 80.6963435594134
                                                   
   4000	min thresh: 80.6963435594134
                                                   
   5000	min thresh: 80.6963435594134
                                                   
   6000	min thresh: 

   7000	min thresh: 6.01161589653171
                                                   
   8000	min thresh: 3.94809218200731
                                                   
   9000	min thresh: 3.94809218200731
                                                   
stepmax	min thresh: 3.94809218200731

hidden: 30    thresh: 0.01    rep: 29/30    steps: 
   1000	min thresh: 11.6541786051526
                                                   
   2000	min thresh: 1.33004095814898
                                                   
   3000	min thresh: 1.26210845076437
                                                   
   4000	min thresh: 1.26210845076437
                                                   
   5000	min thresh: 1.26210845076437
                                                   
   6000	min thresh: 1.26210845076437
                                                   
   7000	min thresh: 1.26210845076437
                                                   
   8000	min thresh: 

 Q3) Build a deep NN model with multiple hidden layers (of your choice) and sigmoid activation function.

In [30]:
# Train the neural network model
nn_md <- neuralnet(bank_formula,          # The formula
                   train.data,            # Training data
                   hidden=c(10, 15),  # Size of the hidden layers
                   #stepmax=10000,        # Maximum training step before stopping
                   rep=1,                 # Number of training repeat
                   lifesign='full',       # Print during train
                   algorithm='backprop',  # Algorithm to calculate the network (backpropagation)
                   learningrate=0.01,     # Learning rate
                   err.fct='ce',          # Error function, cross-entropy
                   act.fct="logistic",    # Function use to calculate the result (activation function)
                   linear.output=F        # FALSE = apply activation function to output layer
                  )

hidden: 10, 15    thresh: 0.01    rep: 1/1    steps: 
   1000	min thresh: 3.73607368209947
                                                     
   2000	min thresh: 2.18222139460499
                                                     
   3000	min thresh: 0.22169253490436
                                                     
   4000	min thresh: 0.22169253490436
                                                     
   5000	min thresh: 0.22169253490436
                                                     
   6000	min thresh: 0.22169253490436
                                                     
   7000	min thresh: 0.22169253490436
                                                     
   8000	min thresh: 0.22169253490436
                                                     
   9000	min thresh: 0.22169253490436
                                                     
  10000	min thresh: 0.0541535511939233
                                                     
  11000	min thresh: 0.054153551193

  89000	min thresh: 0.0258669841293353
                                                     
  90000	min thresh: 0.0258669841293353
                                                     
  91000	min thresh: 0.0258669841293353
                                                     
  92000	min thresh: 0.0258669841293353
                                                     
  93000	min thresh: 0.0258669841293353
                                                     
  94000	min thresh: 0.0258669841293353
                                                     
  95000	min thresh: 0.0258669841293353
                                                     
  96000	min thresh: 0.0258669841293353
                                                     
  97000	min thresh: 0.0258669841293353
                                                     
  98000	min thresh: 0.0258669841293353
                                                     
  99000	min thresh: 0.0258669841293353
                               